In [29]:
from pyspark.sql import SparkSession
import psycopg2
import pandas as pd
import os
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark import SparkConf

# initialise sparkContext

spark = SparkSession.builder.config("spark.jars", "postgresql-42.2.5.jar")\
    .config("spark.jars.packages","org.postgresql:postgresql:42.2.5")\
    .config('spark.driver.maxResultSize','20g').config("spark.executor.memory", "20g")\
    .config("spark.driver.memory","20g").config("spark.executor.memory","20g")\
    .config("spark.shuffle.file.buffer",'40').config("spark.scheduler.listenerbus.eventqueue.capacity","10000000")\
    .master("local").appName("tinkof-invest").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.sparkContext.setLogLevel("ERROR")
spark


In [2]:
hour = spark.createDataFrame(pd.read_parquet('all_candles_hour.parquet'))

In [9]:
min_15 = spark.createDataFrame(pd.read_parquet('all_candles_15min.parquet'))

In [11]:
min_5 = spark.createDataFrame(pd.read_parquet('all_candles_5min.parquet'))

In [13]:
min_1 = spark.createDataFrame(pd.read_parquet('all_candles_1min.parquet'))

In [25]:
mode = "append"
url = "jdbc:postgresql://localhost:5432/shares"
properties = {"user": "nikolay","password": "Andone1","driver": "org.postgresql.Driver"}


In [6]:
hour

DataFrame[figi: string, time: timestamp, open: decimal(14,8), close: decimal(14,8), high: decimal(14,8), low: decimal(14,8), volume: double, is_complete: double]

In [7]:
hour.write.jdbc(url=url, table="candles_hour", mode=mode, properties=properties)

In [10]:
min_15.write.jdbc(url=url, table="candles_15min", mode=mode, properties=properties)

In [12]:
min_5.write.jdbc(url=url, table="candles_5min", mode=mode, properties=properties)

In [14]:
min_1.write.jdbc(url=url, table="candles_1min", mode=mode, properties=properties)

In [9]:
pd.read_parquet('debug/BBG000Q7GG57_hour_from_2022-02-21_to2022-05-20.parquet')

,figi,time,open,close,high,low,volume,is_complete
0,None,2022-02-22 07:00:00+00:00,0.00364500,0.00340000,0.00364500,0.00300500,954,True
1,None,2022-02-22 08:00:00+00:00,0.00339500,0.00337000,0.00349000,0.00332500,293,True
2,None,2022-02-22 09:00:00+00:00,0.00341500,0.00343500,0.00344000,0.00336500,56,True
3,None,2022-02-22 10:00:00+00:00,0.00344000,0.00346500,0.00346500,0.00340000,26,True
4,None,2022-02-22 11:00:00+00:00,0.00344000,0.00345500,0.00349000,0.00344000,39,True
...,...,...,...,...,...,...,...,...
4,None,2022-05-19 11:00:00+00:00,0.00356000,0.00356000,0.00356000,0.00356000,2,True
5,None,2022-05-19 12:00:00+00:00,0.00356500,0.00356500,0.00356500,0.00356500,13,True
6,None,2022-05-19 13:00:00+00:00,0.00355000,0.00356500,0.00356500,0.00355000,10,True
7,None,2022-05-19 14:00:00+00:00,0.00355000,0.00353500,0.00356000,0.00353500,43,True


In [10]:
import os

In [19]:
PATH = os.getcwd()

In [20]:
files = os.listdir(PATH+'/debug')

In [33]:
from tqdm.auto import tqdm
mode = "append"
url = "jdbc:postgresql://localhost:5432/shares"
properties = {"user": "nikolay","password": "Andone1","driver": "org.postgresql.Driver"}
for f in tqdm(files[4:]):
    if 'hour' in f:
        pd_df = pd.read_parquet(PATH+'/debug/'+f)
        if not pd_df.empty:
            pd_df.is_complete = pd_df.is_complete.apply(int)
            hour = spark.createDataFrame(pd_df)
            hour.write.jdbc(url=url, table="candles_hour_rus", mode=mode, properties=properties)
    elif 'day' in f:
        pd_df = pd.read_parquet(PATH+'/debug/'+f)
        if not pd_df.empty:
            pd_df.is_complete = pd_df.is_complete.apply(int)
            day = spark.createDataFrame(pd_df)
            day.write.jdbc(url=url, table="candles_day_rus", mode=mode, properties=properties)

  0%|          | 0/290 [00:00<?, ?it/s]

#
# A fatal error has been detected by the Java Runtime Environment:
#
#  Internal Error (methodData.cpp:1080), pid=77717, tid=77773
#  Error: ShouldNotReachHere()
#
# JRE version: OpenJDK Runtime Environment (11.0.15+10) (build 11.0.15+10-Ubuntu-0ubuntu0.21.10.1)
# Java VM: OpenJDK 64-Bit Server VM (11.0.15+10-Ubuntu-0ubuntu0.21.10.1, mixed mode, sharing, tiered, compressed oops, g1 gc, linux-amd64)
# Core dump will be written. Default location: Core dumps may be processed with "/usr/share/apport/apport -p%p -s%s -c%c -d%d -P%P -u%u -g%g -- %E" (or dumping to /home/nikolay/Projects/tinkoff-invest/Collect_data/core.77717)
#
# An error report file with more information is saved as:
# /home/nikolay/Projects/tinkoff-invest/Collect_data/hs_err_pid77717.log
#
# If you would like to submit a bug report, please visit:
#   Unknown
#


In [32]:
for f in tqdm(files[:10]):
    pd_df = pd.read_parquet(PATH+'/debug/'+f)
    print(pd_df)

  0%|          | 0/10 [00:00<?, ?it/s]

            figi                      time        open       close  \
0   BBG003BNWBP3 2022-02-22 07:00:00+00:00  1.85000000  1.71500000   
1   BBG003BNWBP3 2022-02-22 08:00:00+00:00  1.71400000  1.68500000   
2   BBG003BNWBP3 2022-02-22 09:00:00+00:00  1.66500000  1.69900000   
3   BBG003BNWBP3 2022-02-22 10:00:00+00:00  1.66700000  1.78900000   
4   BBG003BNWBP3 2022-02-22 11:00:00+00:00  1.78100000  1.74600000   
..           ...                       ...         ...         ...   
4   BBG003BNWBP3 2022-05-19 11:00:00+00:00  2.00400000  2.00000000   
5   BBG003BNWBP3 2022-05-19 12:00:00+00:00  1.99500000  2.01400000   
6   BBG003BNWBP3 2022-05-19 13:00:00+00:00  2.01400000  2.00900000   
7   BBG003BNWBP3 2022-05-19 14:00:00+00:00  2.00900000  2.00800000   
8   BBG003BNWBP3 2022-05-19 15:00:00+00:00  2.01300000  2.01100000   

          high         low  volume  is_complete  
0   1.88800000  1.50000000   18136         True  
1   1.77700000  1.63000000    4450         True  
2   1.725